<a href="https://colab.research.google.com/github/sowmya-subu/slay-my-text/blob/main/slaymytext_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from getpass import getpass
from huggingface_hub import login

hf_token = getpass("Enter your Hugging Face token:")
login(token=hf_token)


Enter your Hugging Face token:··········


In [3]:
#Step 1: Install dependencies
!pip install -q bitsandbytes transformers accelerate

# Step 2: Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Step 3: Load model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",              # Automatically use GPU if available
    load_in_4bit=True,              # Efficient 4-bit quantization for free Colab GPU
    torch_dtype=torch.float16
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [1]:
# Step 4: Define simple generation function
def generate_genz_reword(text):
    prompt = f"<|system|>\nYou are a playful, witty Gen Z content creator. Rewrite any input text using Gen Z slang, emojis, and humor while keeping the original meaning.\n\n<|user|>\nRewrite the following text in a Gen Z tone: \"" + text + "\"\n\n<|assistant|>"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response part
    genz_text = response.split('<|assistant|>')[-1].strip()

    # Define basic unsafe words (expand as needed)
    unsafe_words = ["kill", "hate", "suicide", "violence", "racist", "sex", "murder", "abuse"]

    # Check for unsafe content
    if any(word.lower() in genz_text.lower() for word in unsafe_words):
        #log_unsafe(prompt, genz_text)
        return "⚠️ Sorry, I can't reword this text due to unsafe content."

    return genz_text



In [ ]:
# Step 5: Test it!
sample_input = "I'm looking forward to our meeting tomorrow. Let me know if you need anything beforehand."
print("🎤 Original:", sample_input)
print("🎧 Gen Z Style:", generate_genz_reword(sample_input))


In [3]:
!pip install gradio --quiet
import gradio as gr
from datetime import datetime
def capture_feedback(original, reworded, feedback, comment):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("feedback.txt", "a") as f:
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Original: {original}\n")
        f.write(f"Reworded: {reworded}\n")
        f.write(f"Feedback: {feedback}\n")
    return "✅ Feedback saved!"

custom_theme = gr.themes.Soft(
    primary_hue="indigo",
    secondary_hue="pink",
    font=[gr.themes.GoogleFont("Fugaz One"), "cursive"],
    radius_size="lg", # Changed from "xl" to "lg"
    spacing_size="lg"
)

demo = gr.Interface(
    fn=generate_genz_reword,
    inputs="text",
    outputs="text",
    title="SlayMyText - Gen Z Rewording",
    description="Rewrite boring text into slay Gen Z speak",
)

with gr.Blocks(theme=custom_theme) as demo:
    gr.Markdown("## 🎤 SlayMyText: Translate Anything to Gen Z 🔥")

    with gr.Row():
        input_text = gr.Textbox(lines=3, label="Enter Your Text")
        output_text = gr.Textbox(lines=3, label="Fire Text")

    run_btn = gr.Button("✨ Slay it, Don't Say it!")

    run_btn.click(fn=generate_genz_reword, inputs=input_text, outputs=output_text)

    gr.Markdown("### 🙏 Did we slay it or nah?")

    with gr.Row():
        feedback = gr.Radio(choices=["🔥 Slayed", "Mid"], label="Your Reaction")
        comment = gr.Textbox(lines=2, label="Optional Comments (e.g., why it's mid)")
        submit_btn = gr.Button("Submit Feedback")
        status = gr.Textbox(label="", interactive=False)

    submit_btn.click(fn=capture_feedback,
                 inputs=[input_text, output_text, feedback, comment],
                 outputs=status)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e73cbfe230896d9ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
!pip install gradio --quiet
import gradio as gr
import csv
import time
import os

# Define the path for the feedback CSV
FEEDBACK_CSV_PATH = "genz_reword_feedback.csv"

def log_feedback(text, reworded_text, feedback_type):
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    # A simple placeholder for user info - in a real app, you'd get this from authentication
    user_info = "anonymous_user"

    # Append feedback to the CSV file
    with open(FEEDBACK_CSV_PATH, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write header if file is new
        if os.stat(FEEDBACK_CSV_PATH).st_size == 0:
            csv_writer.writerow(["Timestamp", "User Info", "Original Text", "Reworded Text", "Feedback Type"])
        csv_writer.writerow([timestamp, user_info, text, reworded_text, feedback_type])

    print("Feedback logged successfully!")
    return "Feedback submitted! Thanks for helping improve this!"

# Wrap the reword function to log input and output before feedback
def reword_and_log(text):
    reworded_text = generate_genz_reword(text) # Call your existing function
    # Store the original and reworded text temporarily for the feedback function
    # A more robust solution might use session state if available
    # For this example, we'll rely on the Gradio's implicit state passing for flagging
    return reworded_text


demo = gr.Interface(
    fn=reword_and_log,
    inputs="text",
    outputs="text",
    title="Gen Z Rewording with Feedback",
    description="Rewrite your text in Gen Z slang and provide feedback!",
    flagging_enabled=True,
    # Use a custom flagging callback to log to CSV
    # Gradio's default flagging passes input, output, and flag
    flagging_callback=log_feedback,
    flagging_labels=["👍 Good", "👎 Bad", "🤨 Confusing"], # Custom feedback labels
)

demo.launch(share=True)

TypeError: BlockContext.__init__() got an unexpected keyword argument 'flagging_enabled'

In [ ]:
import time

last_request_time = 0

def rate_limited_slay(text):
    global last_request_time
    if time.time() - last_request_time < 10:  # 10-second delay
        return "Please wait a few seconds between rewordings."
    last_request_time = time.time()
    return slay_my_text(text)


In [ ]:
UNSAFE_KEYWORDS = ["kill", "suicide", "hate", "racist"]

def safe_response(text):
    for word in UNSAFE_KEYWORDS:
        if word in text.lower():
            return "⚠️ Content was flagged as unsafe. Please try a different input."
    return text


In [ ]:
def slay_safe(text):
    response = rate_limited_slay(text)
    return safe_response(response)


In [ ]:
!pip install -U bitsandbytes -q